<a href="https://colab.research.google.com/github/frank097/IA-sis420/blob/main/Pregunta2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [197]:
#Nombre : Franklin Correa Paco
#Carrera: Ing. de Sistemas
#Ejercicio 2 del examen final
#
#modelo de redes neuronales para un dataset de diabetes de mas de  253000 datos y 22 columnas

In [198]:
import random
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [199]:
class MLP:
    def __init__(self, layers):
        # el MLP es una lista de capas
        self.layers = layers

    def __call__(self, x):
        # calculamos la salida del modelo aplicando
        # cada capa de manera secuencial
        for layer in self.layers:
            x = layer(x)
        return x

In [200]:
class Layer():
    def __init__(self):
        self.params = []
        self.grads = []

    def __call__(self, x):
        # por defecto, devolver los inputs
        # cada capa hará algo diferente aquí
        return x

    def backward(self, grad):
        # cada capa, calculará sus gradientes
        # y los devolverá para las capas siguientes
        return grad

    def update(self, params):
        # si hay parámetros, los actualizaremos
        # con lo que nos de el optimizer
        return

In [201]:
class Linear(Layer):
    def __init__(self, d_in, d_out):
        # pesos de la capa
        self.w = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(d_in+d_out)),
                                  size=(d_in, d_out))
        self.b = np.zeros(d_out)

    def __call__(self, x):
        self.x = x
        self.params = [self.w, self.b]
        # salida del preceptrón
        return np.dot(x, self.w) + self.b

    def backward(self, grad_output):
        # gradientes para la capa siguiente (BACKPROP)
        grad = np.dot(grad_output, self.w.T)
        self.grad_w = np.dot(self.x.T, grad_output)
        # gradientes para actualizar pesos
        self.grad_b = grad_output.mean(axis=0)*self.x.shape[0]
        self.grads = [self.grad_w, self.grad_b]
        return grad

    def update(self, params):
        self.w, self.b = params

In [202]:
#funciones de activacion
class ReLU(Layer):
    def __call__(self, x):
        self.x = x
        return np.maximum(0, x)

    def backward(self, grad_output):
        grad = self.x > 0
        return grad_output*grad

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

class Sigmoid(Layer):
    def __call__(self, x):
        self.x = x
        return sigmoid(x)

    def backward(self, grad_output):
        grad = sigmoid(self.x)*(1 - sigmoid(self.x))
        return grad_output*grad

In [203]:
#Optimizador, stocastic greadent descent
class SGD():
    def __init__(self, net, lr):
        self.net = net
        self.lr = lr

    def update(self):
        for layer in self.net.layers:
            layer.update([
                params - self.lr*grads
                for params, grads in zip(layer.params, layer.grads)
            ])

In [204]:
class Loss():
    def __init__(self, net):
        self.net = net

    def backward(self):
        # derivada de la loss function con respecto
        # a la salida del MLP
        grad = self.grad_loss()
        # BACKPROPAGATION
        for layer in reversed(self.net.layers):
            grad = layer.backward(grad)

class MSE(Loss):#proporciona una implementación del error cuadrático medio (MSE) y su gradiente
    def __call__(self, output, target):
        self.output, self.target = output, target.reshape(output.shape)
        loss = np.mean((self.output - self.target)**2)
        return loss.mean()

    def grad_loss(self):
        return self.output -  self.target

class BCE(Loss):# proporciona una implementación de la Binary Cross-Entropy (BCE) y su gradiente, que son útiles en problemas de clasificación binaria
    def __call__(self, output, target):
        self.output, self.target = output, target.reshape(output.shape)
        loss = - np.mean(self.target*np.log(self.output) - (1 - self.target)*np.log(1 - self.output))
        return loss.mean()

    def grad_loss(self):
        return self.output -  self.target

class CrossEntropy(Loss): #proporciona una implementación de la función de pérdida
    def __call__(self, output, target):
        self.output, self.target = output, target
        logits = output[np.arange(len(output)), target]
        loss = - logits + np.log(np.sum(np.exp(output), axis=-1))
        loss = loss.mean()
        return loss

    def grad_loss(self):
        answers = np.zeros_like(self.output)
        answers[np.arange(len(self.output)), self.target] = 1
        return (- answers + softmax(self.output)) / self.output.shape[0]

In [205]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from sklearn.metrics import accuracy_score

#Carga de dataset
data = pd.read_csv('/content/drive/MyDrive/sis420/examen final/dataset/diabetes_012_health_indicators_BRFSS2015.csv')

In [206]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253680 entries, 0 to 253679
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Diabetes_012          253680 non-null  float64
 1   HighBP                253680 non-null  float64
 2   HighChol              253680 non-null  float64
 3   CholCheck             253680 non-null  float64
 4   BMI                   253680 non-null  float64
 5   Smoker                253680 non-null  float64
 6   Stroke                253680 non-null  float64
 7   HeartDiseaseorAttack  253680 non-null  float64
 8   PhysActivity          253680 non-null  float64
 9   Fruits                253680 non-null  float64
 10  Veggies               253680 non-null  float64
 11  HvyAlcoholConsump     253680 non-null  float64
 12  AnyHealthcare         253680 non-null  float64
 13  NoDocbcCost           253680 non-null  float64
 14  GenHlth               253680 non-null  float64
 15  

In [207]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253680 entries, 0 to 253679
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Diabetes_012          253680 non-null  float64
 1   HighBP                253680 non-null  float64
 2   HighChol              253680 non-null  float64
 3   CholCheck             253680 non-null  float64
 4   BMI                   253680 non-null  float64
 5   Smoker                253680 non-null  float64
 6   Stroke                253680 non-null  float64
 7   HeartDiseaseorAttack  253680 non-null  float64
 8   PhysActivity          253680 non-null  float64
 9   Fruits                253680 non-null  float64
 10  Veggies               253680 non-null  float64
 11  HvyAlcoholConsump     253680 non-null  float64
 12  AnyHealthcare         253680 non-null  float64
 13  NoDocbcCost           253680 non-null  float64
 14  GenHlth               253680 non-null  float64
 15  

In [208]:
x = data.iloc[:, :21]
y = data.iloc[:, 21]
m = y.size
print(x.shape)
print(y.shape)
print(x)
print('---'*20)
print(y)

(253680, 21)
(253680,)
        Diabetes_012  HighBP  HighChol  CholCheck   BMI  Smoker  Stroke  \
0                0.0     1.0       1.0        1.0  40.0     1.0     0.0   
1                0.0     0.0       0.0        0.0  25.0     1.0     0.0   
2                0.0     1.0       1.0        1.0  28.0     0.0     0.0   
3                0.0     1.0       0.0        1.0  27.0     0.0     0.0   
4                0.0     1.0       1.0        1.0  24.0     0.0     0.0   
...              ...     ...       ...        ...   ...     ...     ...   
253675           0.0     1.0       1.0        1.0  45.0     0.0     0.0   
253676           2.0     1.0       1.0        1.0  18.0     0.0     0.0   
253677           0.0     0.0       0.0        1.0  28.0     0.0     0.0   
253678           0.0     1.0       0.0        1.0  23.0     0.0     0.0   
253679           2.0     1.0       1.0        1.0  25.0     0.0     0.0   

        HeartDiseaseorAttack  PhysActivity  Fruits  ...  HvyAlcoholConsump  

In [209]:
# Crea un nuevo DataFrame con los datos modificados
nuevo_data = data.copy()
# Guardar el dataset actualizado en un nuevo archivo
nuevo_data.to_csv('diabetes1.csv', index=False)

In [210]:
data = np.loadtxt("/content/diabetes1.csv", delimiter=',',skiprows=1)
# print(data)
x, y = data[:, :21].astype(int), data[:, 0].astype(int)
x = x.reshape(len(x),21)
print(x.shape)
print(y.shape)

(253680, 21)
(253680,)


In [211]:
x_train, x_test, y_train, y_test = x[:800] , x[800:] , y[:800].astype(np.float64), y[800:].astype(np.float64)
x_train.shape , x_test.shape

y_train.shape , y_test.shape

((800,), (252880,))

In [212]:
def relu(x):
  return np.maximum(0, x)

def reluPrime(x):
  return x > 0

In [213]:
def linear(x):
    return x

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

In [214]:
# Mean Square Error -> usada para regresión (con activación lineal)
def mse(y, y_hat):
    return np.mean((y_hat - y.reshape(y_hat.shape))**2)

# Binary Cross Entropy -> usada para clasificación binaria (con sigmoid)
def bce(y, y_hat):
    return - np.mean(y.reshape(y_hat.shape)*np.log(y_hat) - (1 - y.reshape(y_hat.shape))*np.log(1 - y_hat))

# Cross Entropy (aplica softmax + cross entropy de manera estable) -> usada para clasificación multiclase
def crossentropy(y, y_hat):
    logits = y_hat[np.arange(len(y_hat)),y]
    entropy = - logits + np.log(np.sum(np.exp(y_hat),axis=-1))
    return entropy.mean()

In [215]:
def grad_mse(y, y_hat):
    return y_hat - y.reshape(y_hat.shape)

def grad_bce(y, y_hat):
    return y_hat - y.reshape(y_hat.shape)

def grad_crossentropy(y, y_hat):
    answers = np.zeros_like(y_hat)
    answers[np.arange(len(y_hat)),y] = 1
    return (- answers + softmax(y_hat)) / y_hat.shape[0]

In [216]:
# clase base MLP

class MLP():
  def __init__(self, D_in, H, D_out, loss, grad_loss, activation):
    # pesos de la capa 1
    self.w1, self.b1 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(D_in+H)),
                                  size=(D_in, H)), np.zeros(H)
    # pesos de la capa 2
    self.w2, self.b2 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(H+D_out)),
                                  size=(H, D_out)), np.zeros(D_out)
    self.ws = []
    # función de pérdida y derivada
    self.loss = loss
    self.grad_loss = grad_loss
    # función de activación
    self.activation = activation

  def __call__(self, x):
    # salida de la capa 1
    self.h_pre = np.dot(x, self.w1) + self.b1
    self.h = relu(self.h_pre)
    # salida del MLP
    y_hat = np.dot(self.h, self.w2) + self.b2
    return self.activation(y_hat)

  def fit(self, X, Y, epochs = 100, lr = 0.001, batch_size=None, verbose=True, log_each=1):
    batch_size = len(X) if batch_size == None else batch_size
    batches = len(X) // batch_size
    l = []
    for e in range(1,epochs+1):
        # Mini-Batch Gradient Descent
        _l = []
        for b in range(batches):
            # batch de datos
            x = X[b*batch_size:(b+1)*batch_size]
            y = Y[b*batch_size:(b+1)*batch_size]
            # salida del perceptrón
            y_pred = self(x)
            # función de pérdida
            loss = self.loss(y, y_pred)
            _l.append(loss)
            # Backprop
            dldy = self.grad_loss(y, y_pred)
            grad_w2 = np.dot(self.h.T, dldy)
            grad_b2 = dldy.mean(axis=0)
            dldh = np.dot(dldy, self.w2.T)*reluPrime(self.h_pre)
            grad_w1 = np.dot(x.T, dldh)
            grad_b1 = dldh.mean(axis=0)
            # Update (GD)
            self.w1 = self.w1 - lr * grad_w1
            self.b1 = self.b1 - lr * grad_b1
            self.w2 = self.w2 - lr * grad_w2
            self.b2 = self.b2 - lr * grad_b2
        l.append(np.mean(_l))
        # guardamos pesos intermedios para visualización
        self.ws.append((
            self.w1.copy(),
            self.b1.copy(),
            self.w2.copy(),
            self.b2.copy()
        ))
        if verbose and not e % log_each:
            print(f'Epoch: {e}/{epochs}, Loss: {np.mean(l):.5f}')

  def predict(self, ws, x):
    w1, b1, w2, b2 = ws
    h = relu(np.dot(x, w1) + b1)
    y_hat = np.dot(h, w2) + b2
    return self.activation(y_hat)
  def evaluate(perceptron, x, t = 0.5):
    w = perceptron.ws[-1]
    x = np.c_[np.ones(len(x)), x]
    y = perceptron(w, x)
    return (y > t).astype(np.float64)

In [217]:
# MLP para regresión
class MLPRegression(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, mse, grad_mse, linear)

# MLP para clasificación binaria
class MLPBinaryClassification(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, bce, grad_bce, sigmoid)

# MLP para clasificación multiclase
class MLPClassification(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, crossentropy, grad_crossentropy, linear)

In [218]:
model = MLPBinaryClassification(D_in=21, H=2, D_out=1)
epochs, lr = 120, 0.04

# normalización datos
x_mean = np.mean(x_train)
x_std = np.std(x_train)
x_std = np.nan_to_num(x_std, nan=1.0)


# Calcula x_norm
x_norm = (x_train - x_mean) / x_std

model.fit(x, y, epochs=100, batch_size=50, log_each=10)
#model.fit(x_norm, y_train, epochs, lr, batch_size=1, log_each=10)

Epoch: 10/100, Loss: 0.07983
Epoch: 20/100, Loss: 0.09635
Epoch: 30/100, Loss: 0.10186
Epoch: 40/100, Loss: 0.10461
Epoch: 50/100, Loss: 0.10626
Epoch: 60/100, Loss: 0.10737
Epoch: 70/100, Loss: 0.10815
Epoch: 80/100, Loss: 0.10874
Epoch: 90/100, Loss: 0.10920
Epoch: 100/100, Loss: 0.10957


In [219]:
# últimos pesos encontrados

w = model.ws[-1]
w

(array([[-0.72720198,  0.6711891 ],
        [ 0.04174955, -0.11481103],
        [ 0.09036221, -0.20200782],
        [ 0.03729196, -0.4091694 ],
        [-0.48816982, -0.58592956],
        [ 0.07088762,  0.43413077],
        [-0.70082454,  0.39666759],
        [-0.46727289,  0.36947011],
        [-0.21822293, -0.43684629],
        [ 0.01219007,  0.04897933],
        [ 0.38222955, -0.13000514],
        [-0.00746263, -0.07996517],
        [ 0.01848628, -0.06525765],
        [-0.11091984, -0.11018406],
        [ 0.14255934,  0.29517914],
        [-0.01036856, -0.3088485 ],
        [ 0.17412241,  0.15467023],
        [ 0.17696758,  0.22385186],
        [ 0.11613038, -0.19559391],
        [-0.02582559, -0.461167  ],
        [-0.05155037, -0.2929751 ]]),
 array([ 0.       , -0.0005934]),
 array([[0.27121853],
        [1.18997767]]),
 array([-0.85803269]))

In [220]:
print("Primeros 5 datos de entrada de prueba (X_test):")
print(x_test[:5])

print("Primeras 5 etiquetas de prueba (y_test):")
print(y_test[:5])

Primeros 5 datos de entrada de prueba (X_test):
[[ 0  0  0  1 27  0  0  0  1  0  0  0  1  0  3  0  0  0  0  1  4]
 [ 2  1  1  1 34  1  0  0  1  0  1  0  1  0  4  0  0  0  1  9  5]
 [ 0  0  0  1 32  0  0  0  1  1  1  0  1  0  4  0  0  0  0  8  5]
 [ 0  1  1  1 26  0  0  0  1  0  0  0  0  1  4  0  0  0  0  8  4]
 [ 0  0  0  1 29  0  0  0  0  0  0  0  1  0  1  0  0  0  1  4  4]]
Primeras 5 etiquetas de prueba (y_test):
[0. 2. 0. 0. 0.]


In [221]:
# nuevo punto
x_new = x_test
X_new = [0, 0, 0, 1, 29, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 4, 4]
y_pred = model.predict(w, X_new)
y_pred

array([0.29775054])

In [222]:
def accuracy(y_pred, y):
    return np.sum(y_pred == y) / len(y)

In [223]:
model.fit(x, y, epochs=150, batch_size=50, log_each=10)

#model.fit(X, Y, epochs=50, batch_size=250, verbose=2, validation_split=0.2)

Epoch: 10/150, Loss: 0.11287
Epoch: 20/150, Loss: 0.11287
Epoch: 30/150, Loss: 0.11287
Epoch: 40/150, Loss: 0.11287
Epoch: 50/150, Loss: 0.11287
Epoch: 60/150, Loss: 0.11287
Epoch: 70/150, Loss: 0.11287
Epoch: 80/150, Loss: 0.11287
Epoch: 90/150, Loss: 0.11287
Epoch: 100/150, Loss: 0.11287
Epoch: 110/150, Loss: 0.11287
Epoch: 120/150, Loss: 0.11287
Epoch: 130/150, Loss: 0.11287
Epoch: 140/150, Loss: 0.11287
Epoch: 150/150, Loss: 0.11287


In [224]:
scores = model.evaluate(x,y)
print("%s: %.2f%%" % (scores[1]*100))

TypeError: ignored